In [1]:
import time
import torch, torchvision
from torch.autograd import Variable
import numpy as np
import pandas as pd
import os
import gc
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import datetime

from DBFNet.Model import *
import warnings

### 计算距离函数

In [2]:
def compute_distance_km(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = lat1.astype('float'), lon1.astype('float'), lat2.astype('float'), lon2.astype('float')
    # 批量计算地球上两点间的球面距离
    R = 6371e3  # 地球半径（米）
    phi_1, phi_2 = np.radians(lat1), np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)
    a = np.power(np.sin(delta_phi / 2), 2) + np.cos(phi_1) * np.cos(phi_2) * np.power(np.sin(delta_lambda / 2), 2)
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c / 1000.

### 归一化函数

In [3]:
def data_scaler(x, feature_range=[0, 1]):
    scaler = MinMaxScaler(feature_range=feature_range)
    shape = x.shape
    x = x.reshape(shape[0], -1)
    x = scaler.fit_transform(x)
    x = x.reshape(shape)
    return x, scaler

### 数据集相关类

In [4]:
class TC_PreTrainLoader(Dataset):
    def __init__(self, pretrain_tc):
        self.tc = pretrain_tc

    def __getitem__(self, index):
        return self.tc[index, :, :-2], self.tc[index, :, -2:]

    def __len__(self):
        return self.tc.shape[0]
    
class GPH_PreTrainLoader(Dataset):
    def __init__(self, gph):
        self.g = gph

    def __getitem__(self, item):
        return self.g[item, :4], self.g[item, 4:]

    def __len__(self):
        return len(self.g)

class TrainLoader(Dataset):
    def __init__(self, x, y, g, standard):
        self.x = x
        self.y = y
        self.g = g
        self.s = standard

    def __getitem__(self, item):
        return self.x[item], self.g[item, :4], self.y[item], self.g[item, 4:], self.s[item]

    def __len__(self):
        return len(self.x)

### 训练函数

In [5]:
def train(cfg1=TC_Encoder_config, cfg2=Map_Encoder_config, checkpoint=None,
          tc_pretrain_path='./CMA_dataset/CMA_csv/xtc_pretrain_data.npy', xtc_path='./CMA_dataset/xtc.npy',
          ytc_path='./CMA_dataset/ytc.npy', split_index_path='./CMA_dataset/split_index.npy', gph_path='./gph.npy',
          tc_pretrain_epoch=50, tc_pretrain_lr=0.01, gph_pretrain_epoch=50, gph_pretrain_lr=0.01, batch_size=32,
          epoch=50, lr=0.001, model_save_path='./model/model001.pth'):
    warnings.filterwarnings("ignore") # 关闭warning

    model = DBFNet(cfg1, cfg2)
    if checkpoint is not None:
        state_dict = torch.load(checkpoint)
        model.load_state_dict(state_dict)
    criterion = nn.L1Loss()
    if torch.cuda.is_available():
        model = model.cuda()


    # 载入TC预训练数据
    print('处理TC Encoder预训练数据：', end='')
    pre_tc = np.load(tc_pretrain_path)
    pre_tc, pre_tc_scaler = data_scaler(pre_tc)
    pre_tc = DataLoader(TC_PreTrainLoader(pre_tc), batch_size=batch_size, shuffle=True)
    print('完成')

    # TC Encoder预训练
    print('TC Encoder 预训练：')
    model.train()
    for i in range(tc_pretrain_epoch):
        optimizer = torch.optim.Adam(model.parameters(), lr=tc_pretrain_lr * (0.1 ** (i // 20)))
        # print(tc_pretrain_lr * (0.1 ** (i // 20)))
        time1 = datetime.datetime.now()
        pretrain_loss = 0
        for X, Y in pre_tc:
            if torch.cuda.is_available():
                x = X.float().cuda()
                y = Y.float().cuda()
            else:
                x = X.float()
                y = Y.float()
            optimizer.zero_grad()
            pred = model.pretrain_tc_forward(x)
            loss = criterion(pred, y)
            loss.backward()
            optimizer.step()
            pretrain_loss += loss.detach().item()
        time2 = datetime.datetime.now()
        print('Pretrain TC [%d/%d] cost:%ds train_loss: %.5f' % (
        i + 1, tc_pretrain_epoch, (time2 - time1).seconds, pretrain_loss))
    del pre_tc # 降低内存占用


    # 载入完整数据集
    print('载入完整数据集...')
    xtc = np.load(xtc_path, allow_pickle=True).astype(float)
    ytc = np.load(ytc_path, allow_pickle=True).astype(float)
    split_index = np.load(split_index_path, allow_pickle=True).astype(int)
    gph = np.load(gph_path, allow_pickle=True).astype(float)
    full_train_index = [*range(split_index[0], split_index[2])]
    test_index = [*range(split_index[2], split_index[3])]
    standard = xtc[:, -1, :2].reshape(-1, 1, 2)


    # 归一化
    xtc, _ = data_scaler(xtc)
    # _ , y_scaler = data_scaler(ytc)
    gph, _ = data_scaler(gph)
    # gph, _ = data_scaler(gph)

    # 打包GPH预训练数据
    pre_gph = DataLoader(GPH_PreTrainLoader(gph), batch_size=batch_size, shuffle=True, drop_last=True)

    # GPH Encoder-Decoder预训练
    print('GPH Encoder-Decoder 预训练：')
    model.train()
    for i in range(gph_pretrain_epoch):
        optimizer = torch.optim.Adam(model.parameters(), lr=gph_pretrain_lr * (0.1 ** (i // 20)))
        time1 = datetime.datetime.now()
        pretrain_loss = 0
        for X, Y in tqdm(pre_gph):
            if torch.cuda.is_available():
                x = X.float().cuda()
                y = Y.float().cuda()
            else:
                x = X.float()
                y = Y.float()
            optimizer.zero_grad()
            pred = model.pretrain_map_forward(x)
            loss = criterion(pred, y)
            loss.backward()
            # print([x.grad for x in optimizer.param_groups[0]['params']])
            optimizer.step()
            pretrain_loss += loss.detach().item()
        time2 = datetime.datetime.now()
        print('Pretrain GPH [%d/%d] cost:%ds train_loss: %.5f' % (i + 1, gph_pretrain_epoch, (time2 - time1).seconds,
                                                                  pretrain_loss))
    del pre_gph # 降低内存占用


    # 从训练数据中划分训练集和验证集
    train_index, val_index, _, _ = train_test_split(full_train_index, full_train_index, test_size=0.2)
    # 打包数据集
    train_dataset = DataLoader(TrainLoader(xtc[train_index], ytc[train_index], gph[train_index], standard[train_index]),
                               batch_size=batch_size, shuffle=True)
    val_dataset = DataLoader(TrainLoader(xtc[val_index], ytc[val_index], gph[val_index], standard[val_index]),
                             batch_size=batch_size, shuffle=True)
    test_dataset = DataLoader(TrainLoader(xtc[test_index], ytc[test_index], gph[test_index], standard[test_index]),
                              batch_size=batch_size, shuffle=False)
    del xtc, ytc, gph, split_index # 降低内存占用

    # 整个模型训练
    print('完整模型训练：')
    for i in range(epoch):
        optimizer = torch.optim.Adam(model.parameters(), lr=lr * (0.1 ** (i // 20)))
        # 训练集
        time1 = datetime.datetime.now()
        model.train()
        train_loss1, train_loss2 = 0, 0
        for X1, X2, Y1, Y2, _ in tqdm(train_dataset):
            if torch.cuda.is_available():
                x1 = X1.float().cuda()
                x2 = X2.float().cuda()
                y1 = Y1.float().cuda()
                y2 = Y2.float().cuda()
            else:
                x1 = X1.float()
                x2 = X2.float()
                y1 = Y1.float()
                y2 = Y2.float()
            optimizer.zero_grad()
            pred1, pred2 = model(x1, x2)
            loss1 = criterion(pred1, y1)
            loss2 = criterion(pred2, y2)
            loss = loss1 + loss2
            loss.backward()
            optimizer.step()
            train_loss1 += loss1.detach().item()
            train_loss2 += loss2.detach().item()
        torch.save(model.state_dict(), model_save_path)
        time2 = datetime.datetime.now()
        print('[%d/%d] Train cost:%ds loss: %.5f/%.5f' % (i + 1, epoch, (time2 - time1).seconds, train_loss1, train_loss2), end=' ')
        # 验证集
        model.eval()
        with torch.no_grad():
            time1 = datetime.datetime.now()
            val_loss1, val_loss2 = 0, 0
            for X1, X2, Y1, Y2, _ in val_dataset:
                if torch.cuda.is_available():
                    x1 = X1.float().cuda()
                    x2 = X2.float().cuda()
                    y1 = Y1.float().cuda()
                    y2 = Y2.float().cuda()
                else:
                    x1 = X1.float()
                    x2 = X2.float()
                    y1 = Y1.float()
                    y2 = Y2.float()
                pred1, pred2 = model(x1, x2)
                loss1 = criterion(pred1, y1)
                loss2 = criterion(pred2, y2)
                val_loss1 += loss1.detach().item()
                val_loss2 += loss2.detach().item()
            time2 = datetime.datetime.now()
        print('  ||  Valid cost:%ds loss: %.5f/%.5f' % ((time2 - time1).seconds, val_loss1, val_loss2))
    print('训练完成')

    # 在测试集上预测
    print('测试集上预测：')
    model.eval()
    pred_save, truth_save, standard_save = [], [], []
    for X1, X2, Y1, _, S in tqdm(test_dataset):
        if torch.cuda.is_available():
            x1 = X1.float().cuda()
            x2 = X2.float().cuda()
            y1 = Y1.float().cuda()
            s = S.float().cuda()
        else:
            x1 = X1.float()
            x2 = X2.float()
            y1 = Y1.float()
            s = S.float()
        pred, _ = model(x1, x2)
        pred_save.append(pred.detach().cpu().numpy())
        truth_save.append(y1.detach().cpu().numpy())
        standard_save.append(s.detach().cpu().numpy())
    # 做反归一化
    pred_save = np.concatenate(pred_save, axis=0)
    truth_save = np.concatenate(truth_save, axis=0)
    standard_save = np.concatenate(standard_save, axis=0)
    # pred_save = data_scaler_inverse(pred_save, y_scaler)
    # 计算最终预测结果
    pred = pred_save + standard_save
    truth = truth_save + standard_save
    for i in range(1, 5):
        pred_lat, pred_lon, truth_lat, truth_lon = pred[:, i - 1, 0], pred[:, i - 1, 1], truth[:, i - 1, 0], truth[:, i - 1, 1]
        distance = compute_distance_km(pred_lat, pred_lon, truth_lat, truth_lon)
        print(str(i * 6) + 'h APE：', distance.mean())

### 训练

In [6]:
train(model_save_path='./model/model_demo.pth')
gc.collect()

处理TC Encoder预训练数据：完成
TC Encoder 预训练：
Pretrain TC [1/50] cost:1s train_loss: 16.89595
Pretrain TC [2/50] cost:1s train_loss: 5.94226
Pretrain TC [3/50] cost:1s train_loss: 5.21437
Pretrain TC [4/50] cost:1s train_loss: 4.73386
Pretrain TC [5/50] cost:1s train_loss: 4.44967
Pretrain TC [6/50] cost:1s train_loss: 4.24450
Pretrain TC [7/50] cost:1s train_loss: 3.96177
Pretrain TC [8/50] cost:1s train_loss: 4.01570
Pretrain TC [9/50] cost:1s train_loss: 3.90488
Pretrain TC [10/50] cost:1s train_loss: 3.81908
Pretrain TC [11/50] cost:1s train_loss: 3.81404
Pretrain TC [12/50] cost:1s train_loss: 3.70426
Pretrain TC [13/50] cost:1s train_loss: 3.70496
Pretrain TC [14/50] cost:1s train_loss: 3.61890
Pretrain TC [15/50] cost:1s train_loss: 3.60796
Pretrain TC [16/50] cost:1s train_loss: 3.51278
Pretrain TC [17/50] cost:1s train_loss: 3.60853
Pretrain TC [18/50] cost:1s train_loss: 3.36293
Pretrain TC [19/50] cost:1s train_loss: 3.43408
Pretrain TC [20/50] cost:1s train_loss: 3.26364
Pretrain TC

100%|██████████| 781/781 [00:30<00:00, 25.86it/s]


Pretrain GPH [1/50] cost:30s train_loss: 49.09777


100%|██████████| 781/781 [00:31<00:00, 24.43it/s]


Pretrain GPH [2/50] cost:31s train_loss: 38.30277


...

100%|██████████| 533/533 [00:27<00:00, 19.71it/s]


[50/50] Train cost:27s loss: 328.11462/18.43408   ||  Valid cost:2s loss: 83.54688/4.60808
训练完成
测试集上预测：


100%|██████████| 86/86 [00:01<00:00, 53.18it/s]


6h APE： 38.589675380111544
12h APE： 81.67666828407347
18h APE： 131.1015203232273
24h APE： 188.08918189553893


66

In [7]:
def test_2018_2021(cfg1=TC_Encoder_config, cfg2=Map_Encoder_config, checkpoint_folder='./model/',
                   xtc_path='./CMA_dataset/xtc.npy', ytc_path='./CMA_dataset/ytc.npy', gph_path='./gph.npy',
                   batch_size=32, is_save=True, checkpoint='./model/model_demo.pth'):
    warnings.filterwarnings("ignore")  # 关闭warning

    # 载入完整数据集
    print('载入完整数据集...')
    xtc = np.load(xtc_path, allow_pickle=True).astype(float)
    ytc = np.load(ytc_path, allow_pickle=True).astype(float)
    gph = np.load(gph_path, allow_pickle=True).astype(float)
    test_index = [*range(22227, 24614)] # 2018年至2021年的台风数据索引
    standard = xtc[:, -1, :2].reshape(-1, 1, 2)

    # 归一化
    xtc, _ = data_scaler(xtc)
    gph, _ = data_scaler(gph)

    # 打包数据集
    test_dataset = DataLoader(TrainLoader(xtc[test_index], ytc[test_index], gph[test_index], standard[test_index]),
                              batch_size=batch_size, shuffle=False)
    del xtc, ytc, standard, gph

    model = DBFNet(cfg1, cfg2)
    if torch.cuda.is_available():
        model = model.cuda()
    state_dict = torch.load(checkpoint)
    model.load_state_dict(state_dict)

    # 在测试集上预测
    print('测试集上预测：')
    model.eval()
    pred_save, truth_save, standard_save = [], [], []
    for X1, X2, Y1, _, S in tqdm(test_dataset):
        if torch.cuda.is_available():
            x1 = X1.float().cuda()
            x2 = X2.float().cuda()
            y1 = Y1.float().cuda()
            s = S.float().cuda()
        else:
            x1 = X1.float()
            x2 = X2.float()
            y1 = Y1.float()
            s = S.float()
        pred, _ = model(x1, x2)
        pred_save.append(pred.detach().cpu().numpy())
        truth_save.append(y1.detach().cpu().numpy())
        standard_save.append(s.detach().cpu().numpy())
    # 做反归一化
    pred_save = np.concatenate(pred_save, axis=0)
    truth_save = np.concatenate(truth_save, axis=0)
    standard_save = np.concatenate(standard_save, axis=0)
    # pred_save = data_scaler_inverse(pred_save, y_scaler)
    # 计算最终预测结果
    pred = pred_save + standard_save
    truth = truth_save + standard_save
    for i in range(1, 5):
        pred_lat, pred_lon, truth_lat, truth_lon = pred[:, i - 1, 0], pred[:, i - 1, 1], truth[:, i - 1, 0], truth[:,
                                                                                                             i - 1, 1]
        distance = compute_distance_km(pred_lat, pred_lon, truth_lat, truth_lon)
        print(str(i * 6) + 'h APE：', distance.mean())

In [8]:
test_2018_2021()

载入完整数据集...
测试集上预测：


100%|██████████| 75/75 [00:01<00:00, 66.78it/s]

6h APE： 39.73501210493949
12h APE： 83.73567743841966
18h APE： 133.86184621412266
24h APE： 190.3670649653396
